In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

np.set_printoptions(precision=3, linewidth=120)

In [2]:
# Spin-unresolved operator string with default phase factor 1.
operator_string = gqcpy.SpinUnresolvedOperatorString([0, 2, 3, 1, 5, 4])

assert operator_string.phaseFactor() == 1, "The phase factor is not equal to 1."

### Sort the `SpinUnresolvedOperatorString`

In [3]:
operator_string.sort()

print("The sorted operator string: {} with phase factor {}.".format(operator_string.operatorIndices(), operator_string.phaseFactor()))

The sorted operator string: [0, 1, 2, 3, 4, 5] with phase factor -1.


### Partition the sorted `SpinUnresolvedOperatorString` into two subsystems

In [4]:
partition = ['I', 'J', 'I', 'I', 'J', 'J']

system, environment = operator_string.schmidtDecomposition(partition)

print("The system consists of sites: {} and has phase factor {}.".format(system.operatorIndices(), system.phaseFactor()))
print("The environment consists of sites: {} and has phase factor {}.".format(environment.operatorIndices(), environment.phaseFactor()))

The system consists of sites: [0, 2, 3] and has phase factor -1.
The environment consists of sites: [1, 4, 5] and has phase factor 1.


### Partitioning of a `SpinResolvedONV` into two subsystems

In [5]:
onv_sr = gqcpy.SpinResolvedONV.UHF(6, 4, 2)

operator_string_alpha = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_sr.onv(gqcpy.Spin.alpha))
operator_string_beta = gqcpy.SpinUnresolvedOperatorString.FromONV(onv_sr.onv(gqcpy.Spin.beta))

print("The alpha string consists of sites: {} and has phase factor {}.".format(operator_string_alpha.operatorIndices(), operator_string_alpha.phaseFactor()))
print("The beta string consists of sites: {} and has phase factor {}.".format(operator_string_beta.operatorIndices(), operator_string_beta.phaseFactor()))

The alpha string consists of sites: [0, 1, 2, 3] and has phase factor 1.
The beta string consists of sites: [0, 1] and has phase factor 1.


In [19]:
sites_partitioning = np.array(['J', 'I', 'J', 'I', 'I', 'I'])# 'I' = system, 'J' = environment.
K_system = np.count_nonzero(sites_partitioning == 'I')
K_environment = np.count_nonzero(sites_partitioning == 'J')

alpha_partition = sites_partitioning[operator_string_alpha.operatorIndices()]
beta_partition = sites_partitioning[operator_string_beta.operatorIndices()]

In [20]:
system_alpha, environment_alpha = operator_string_alpha.schmidtDecomposition(alpha_partition)
system_beta, environment_beta = operator_string_beta.schmidtDecomposition(beta_partition)

In [21]:
print("The alpha system consists of sites: {} and has phase factor {}.".format(system_alpha.operatorIndices(), system_alpha.phaseFactor()))
print("The alpha environment consists of sites: {} and has phase factor {}.\n".format(environment_alpha.operatorIndices(), environment_alpha.phaseFactor()))

print("The beta system consists of sites: {} and has phase factor {}.".format(system_beta.operatorIndices(), system_beta.phaseFactor()))
print("The beta environment consists of sites: {} and has phase factor {}.".format(environment_beta.operatorIndices(), environment_beta.phaseFactor()))

The alpha system consists of sites: [1, 3] and has phase factor -1.
The alpha environment consists of sites: [0, 2] and has phase factor 1.

The beta system consists of sites: [1] and has phase factor -1.
The beta environment consists of sites: [0] and has phase factor 1.


In [46]:
def reduce_indices(indices_A, indices_B):
    
    indices_A, indices_B = np.array(indices_A), np.array(indices_B)
    new_indices_A, new_indices_B = [], []
    
    
    for i in indices_A:
        new_indices_A.append( i - np.count_nonzero(indices_B < i) )
    for i in indices_B:
        new_indices_B.append( i - np.count_nonzero(indices_A < i) )
    
    return new_indices_A, new_indices_B

In [47]:
system_alpha_indices, environment_alpha_indices = reduce_indices(system_alpha.operatorIndices(), environment_alpha.operatorIndices())
system_beta_indices, environment_beta_indices = reduce_indices(system_beta.operatorIndices(), environment_beta.operatorIndices())

onv_system_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_alpha_indices, K_system)
onv_system_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(system_beta_indices, K_system)
onv_system = gqcpy.SpinResolvedONV(onv_system_alpha, onv_system_beta)

print("The spin-resolved ONV of the system is: {}".format(onv_system))
# Note: read ONVs from right to left due to bitstring representation.

onv_environment_alpha = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_alpha_indices, K_environment)
onv_environment_beta = gqcpy.SpinUnresolvedONV.FromOccupiedIndices(environment_beta_indices, K_environment)

onv_environment = gqcpy.SpinResolvedONV(onv_environment_alpha, onv_environment_beta)

print("The spin-resolved ONV of the environment is: {}".format(onv_environment))
# Note: read ONVs from right to left due to bitstring representation.

The spin-resolved ONV of the system is: 0011|0001
The spin-resolved ONV of the environment is: 11|01
